$
\newcommand{\red}[1]{{\color{red}{#1}}}
\newcommand{\green}[1]{{\color{green}{#1}}}
\newcommand{\blue}[1]{{\color{blue}{#1}}}
\newcommand{\grey}[1]{{\color{grey}{#1}}}
\newcommand{\orange}[1]{{\color{orange}{#1}}}
\newcommand{\msbr}[1]{\begin{bmatrix}#1\end{bmatrix}}
$

In [ ]:
%pip install -q ipympl
%matplotlib widget

In [ ]:
import math

import numpy as np

pi = np.pi
sqrt = np.sqrt
dot = lambda x, y: x.dot(y)
norm = lambda x: np.linalg.norm(x)

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from IPython.display import display, Latex, Markdown

import ipywidgets as widgets
from ipywidgets import FloatSlider, Label, Output, Layout

try:
    import google.colab
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

def vec(*args):
    if len(args) == 1:
        return np.array(args[0], dtype=np.float64)
    else:
        return np.array(args, dtype=np.float64)

def vec_to_str(vec, row=True, fmt=".2g"):
    if row:
        return "\\begin{bmatrix} " + " \\\\ ".join((f"{{:{fmt}}}").format(val) for val in vec) + " \\end{bmatrix}"
    else:
        return "\\begin{bmatrix} " + " & ".join((f"{{:{fmt}}}").format(val) for val in vec) + " \\end{bmatrix}"

def vec_scale(vect, eps=0.1):
    return vect * (1.0 + eps/(np.linalg.norm(vect) + 0.01))
def vec_scale_2d(vect, off, eps=1.0, xlim=(-4.0, 4.0), ylim=(-4.0, 4.0)):
    return off + vect * (1.0 + eps/(np.linalg.norm(vect) + 0.01) * vec(xlim[1]-xlim[0], ylim[1]-ylim[0]))
def vec_scale_3d(vect, off, eps=0.1, xlim=(-2.0, 2.0), ylim=(-2.0, 2.0), zlim=(-2.0, 2.0)):
    return off + vect * (1.0 + eps/(np.linalg.norm(vect) + 0.01) * vec(xlim[1]-xlim[0], ylim[1]-ylim[0], zlim[1]-zlim[0]))
def vec_ratio(vect, rat=0.2):
    return rat / (np.linalg.norm(vect) + 0.01)
def vec_ratio_3d(vect, rat=0.2, xlim=(-2.0, 2.0), ylim=(-2.0, 2.0), zlim=(-2.0, 2.0)):
    return rat / (np.linalg.norm(vect) + 0.01) * norm(vec(xlim[1]-xlim[0], ylim[1]-ylim[0], zlim[1]-zlim[0]))

def draw_cube(ax):
    for y in [-1.0, 1.0]:
        for z in [-1.0, 1.0]:
            ax.plot([-1.0, 1.0], [y, y], [z, z], color="black", linewidth=0.5, linestyle="-")
    for x in [-1.0, 1.0]:
        for z in [-1.0, 1.0]:
            ax.plot([x, x], [-1.0, 1.0], [z, z], color="black", linewidth=0.5, linestyle="-")
    for x in [-1.0, 1.0]:
        for y in [-1.0, 1.0]:
            ax.plot([x, x], [y, y], [-1.0, 1.0], color="black", linewidth=0.5, linestyle="-")

def draw_line(ax, coefs, xlim=(-4.0, 4.0), ylim=(-4.0, 4.0)):
    a, b, c = coefs
    if np.abs(a) > np.abs(b):
        y = np.linspace(*ylim, 21)
        x = (c/a) - (b/a)*y
        x[(x > xlim[1]) | (x < xlim[0])] = np.nan
    else:
        x = np.linspace(*xlim, 21)
        y = (c/b) - (a/b)*x
        y[(y > ylim[1]) | (y < ylim[0])] = np.nan
    ax.plot(x, y, color="orange", linewidth=1.0, zorder=5)

def draw_plane(ax, coefs, xlim=(-2.0, 2.0), ylim=(-2.0, 2.0), zlim=(-2.0, 2.0)):
    a, b, c, d = coefs
    if np.abs(a) > np.abs(b) and np.abs(a) > np.abs(c):
        (y, z) = np.meshgrid(np.linspace(*ylim, 21), np.linspace(*zlim, 21))
        x = (d/a) - (b/a)*y - (c/a)*z
        x[(x > xlim[1]) | (x < xlim[0])] = np.nan
    elif np.abs(b) > np.abs(a) and np.abs(b) > np.abs(c):
        (x, z) = np.meshgrid(np.linspace(*xlim, 21), np.linspace(*zlim, 21))
        y = (d/b) - (a/b)*x - (c/b)*z
        y[(y > ylim[1]) | (y < ylim[0])] = np.nan
    else:
        (x, y) = np.meshgrid(np.linspace(*xlim, 21), np.linspace(*ylim, 21))
        z = (d/c) - (a/c)*x - (b/c)*y
        z[(z > zlim[1]) | (z < zlim[0])] = np.nan
    ax.plot_surface(x, y, z, color="orange", alpha=0.3)

In [ ]:
u = vec(2.0, 1.0)

n = vec(-1.0, 2.0)

o = vec(0.0, 0.0)
# o = vec(1.0, 1.0)

s = -1.0
w = s*u

# a, b, c = -1.0, 2.0, 1.0
a, b, c = -1.0, 2.0, 0.0

xlim = (-4.0, 4.0)
ylim = (-4.0, 4.0)

plt.close("all")
with plt.ioff():
    fig = plt.figure(figsize=(5.0, 5.0))
ax = fig.add_subplot(111)
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
ax.set_clip_on(True)
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_aspect("equal")
ax.grid(which="both")

slider_s = FloatSlider(
    orientation="vertical", description="s",
    value=s, min=-2.0, max=2.0, step=-0.01,
    readout_format=".2f", layout=Layout(height=("5in"))
)
def update_s(change):
    global s, w
    s = change.new
    w = s*u
    draw_w_2d(ax, w)
    fig.canvas.draw()
    fig.canvas.flush_events()
slider_s.observe(update_s, names="value")

ax.scatter(0.0, 0.0, s=5.0, color="black", zorder=4)

ax.quiver(*o, *u, color="green", angles="xy", scale_units="xy", scale=1.0, width=0.0025*norm(u), zorder=6)
ax.text(*vec_scale_2d(u, o, 0.02, xlim, ylim), "$\\mathbf{u}$", color="green", va="center", ha="center", zorder=6)

ax.quiver(*o, *n, color="red", angles="xy", scale_units="xy", scale=1.0, width=0.0025*norm(u), zorder=7)
ax.text(*vec_scale_2d(n, o, 0.02, xlim, ylim), "$\\mathbf{n}$", color="red", va="center", ha="center", zorder=7)

info_w_2d = [None, None]
def draw_w_2d(ax, w):
    global info_w_2d
    if info_w_2d[0] is not None:
        info_w_2d[0].remove()
    info_w_2d[0] = ax.quiver(*o, *w, color="black", angles="xy", scale_units="xy", scale=1.0, width=0.0025*norm(u), zorder=8)
    if info_w_2d[1] is not None:
        info_w_2d[1].remove()
        info_w_2d[1] = None
    pos_text = vec_scale_2d(w, o, 0.02, xlim, ylim)
    if xlim[0] < pos_text[0] < xlim[1] and ylim[0] < pos_text[1] < ylim[1]:
        info_w_2d[1] = ax.text(*pos_text, "$\\mathbf{w}$", color="black", va="center", ha="center", zorder=8)
draw_w_2d(ax, w)

draw_line(ax, (a, b, c), xlim, ylim)

# ax.quiver(0.0, 0.0, *o, color="grey", linestyle="--", angles="xy", scale_units="xy", scale=1.0, width=0.0025*norm(u), zorder=6)

text = Output(layout=Layout(width="1.5in"))
with text:
    display(Markdown("$ \\green{\\mathbf{u}} = \\green{ " + vec_to_str(u) + " } $"))
    display(Markdown("$ \\red{\\mathbf{n}} = \\red{ " + vec_to_str(n) + " } $"))
    display(Markdown("$ \\mathbf{w} = s \\green{\\mathbf{u}} $"))
    display(Markdown("$ \\orange{ L : " + "{:.2g}".format(a) + " x + " + "{:.2g}".format(b) + " y = " + "{:.2g}".format(c) + " } $"))

fig.tight_layout()
display(widgets.HBox([widgets.VBox([widgets.Box(layout=Layout(flex="1 1 0%", height="auto")), text, widgets.Box(layout=Layout(flex="1 1 0%", height="auto"))]), slider_s, fig.canvas]))

In [ ]:
u = vec(2.0, -1.0, 0.0)
v = vec(-1.0, 1.0, -1.0)

n = vec(1.0, 2.0, 1.0)

o = vec(0.0, 0.0, 0.0)
# o = vec(1.0, 0.0, 0.0)

a, b, c, d = 1.0, 2.0, 1.0, 0.0
# a, b, c, d = 1.0, 2.0, 1.0, 1.0

s = 1.0
t = 1.0

w = s*u + t*v

xlim = (-2.0, 2.0)
ylim = (-2.0, 2.0)
zlim = (-2.0, 2.0)

plt.close("all")
with plt.ioff():
    fig = plt.figure(figsize=(5.0, 5.0))
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
ax = fig.add_subplot(111, projection="3d")
ax.set_clip_on(False)
ax.set_proj_type("persp", focal_length=0.33)
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)
ax.set_zlim(*zlim)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")
ax.set_aspect("equal")
ax.grid(False)
slider_s = FloatSlider(
    orientation="vertical", description="s",
    value=s, min=-2.0, max=2.0, step=-0.01,
    readout_format=".2f", layout=Layout(height=("5in"))
)
slider_t = FloatSlider(
    orientation="vertical", description="t",
    value=t, min=-2.0, max=2.0, step=-0.01,
    readout_format=".2f", layout=Layout(height=("5in"))
)
def update_s(change):
    global s, w
    s = change.new
    w = s*u + t*v
    draw_w(ax, w)
    fig.canvas.draw()
    fig.canvas.flush_events()
def update_t(change):
    global t, w
    t = change.new
    w = s*u + t*v
    draw_w(ax, w)
    fig.canvas.draw()
    fig.canvas.flush_events()
slider_s.observe(update_s, names="value")
slider_t.observe(update_t, names="value")

ax.scatter(0.0, 0.0, 0.0, color="black", s=5.0)
ax.plot([0.0, xlim[1]], [0.0, 0.0], [0.0, 0.0], color="darkred", linewidth=0.25)
ax.plot([0.0, 0.0], [0.0, ylim[1]], [0.0, 0.0], color="darkgreen", linewidth=0.25)
ax.plot([0.0, 0.0], [0.0, 0.0], [0.0, zlim[1]], color="darkblue", linewidth=0.25)

draw_plane(ax, (a, b, c, d), xlim, ylim, zlim)

ax.quiver(*o, *u, color="green", arrow_length_ratio=vec_ratio_3d(u, 0.03, xlim, ylim, zlim))
ax.text(*vec_scale_3d(u, o, 0.03, xlim, ylim, zlim), "$\\mathbf{u}$", color="green", va="center", ha="center")

ax.quiver(*o, *v, color="blue", arrow_length_ratio=vec_ratio_3d(v, 0.03, xlim, ylim, zlim))
ax.text(*vec_scale_3d(v, o, 0.03, xlim, ylim, zlim), "$\\mathbf{v}$", color="blue", va="center", ha="center")

ax.quiver(*o, *n, color="red", arrow_length_ratio=vec_ratio_3d(n, 0.03, xlim, ylim, zlim))
ax.text(*vec_scale_3d(n, o, 0.03, xlim, ylim, zlim), "$\\mathbf{n}$", color="red", va="center", ha="center")

# ax.quiver(0.0, 0.0, 0.0, *o, color="grey", linestyle="--", arrow_length_ratio=vec_ratio_3d(o, 0.03, xlim, ylim, zlim))

info_w = [None, None]
def draw_w(ax, w):
    global info_w
    if info_w[0] is not None:
        info_w[0].remove()
    info_w[0] = ax.quiver(*o, *w, color="black", arrow_length_ratio=vec_ratio_3d(w, 0.03, xlim, ylim, zlim))
    if info_w[1] is not None:
        info_w[1].remove()
    info_w[1] = ax.text(*vec_scale_3d(w, o, 0.03, xlim, ylim, zlim), "$\\mathbf{w}$", color="black", va="center", ha="center")
draw_w(ax, w)

text = Output(layout=Layout(width="2.0in"))
with text:
    display(Markdown("$ \\green{\\mathbf{u}} = \\green{ " + vec_to_str(u) + " } $"))
    display(Markdown("$ \\green{\\mathbf{v}} = \\blue{ " + vec_to_str(v) + " } $"))
    display(Markdown("$ \\red{\\mathbf{n}} = \\red{ " + vec_to_str(n) + " } $"))
    display(Markdown("$ \\mathbf{w} = s \\green{\\mathbf{u}} + t \\blue{\\mathbf{v}} $"))
    display(Markdown("$ \\orange{ L : " + "{:.2g}".format(a) + " x + " + "{:.2g}".format(b) + " y + " + "{:.2g}".format(c) + " z = " + "{:.2g}".format(d) + " } $"))

fig.tight_layout()
display(widgets.HBox([widgets.VBox([widgets.Box(layout=Layout(flex="1 1 auto", height="auto")), text, widgets.Box(layout=Layout(flex="1 1 0%", height="auto"))]), slider_s, slider_t, fig.canvas]))